#### 1. Establish a connection between Python and the Sakila database.

In [5]:
import pandas as pd # all time remember to import pandas library

psw = '.Aa654321.'

import mysql.connector

cnx = mysql.connector.connect(user = 'root',
                              password = psw,
                              host = 'localhost',
                              database = 'Sakila',
                              auth_plugin = 'mysql_native_password')

cursor = cnx.cursor()

cursor

#### 2.Write a Python function called rentals_month that retrieves rental data for a given month and year (passed as parameters) from the Sakila database as a Pandas DataFrame. The function should take in three parameters:

- engine: an object representing the database connection engine to be used to establish a connection to the Sakila database.
- month: an integer representing the month for which rental data is to be retrieved.
- year: an integer representing the year for which rental data is to be retrieved.

The function should execute a SQL query to retrieve the rental data for the specified month and year from the rental table in the Sakila database, and return it as a pandas DataFrame.

In [45]:
def rentals_month(engine,month,year):
    # first of all we define the query we will do
    query = f"""
    SELECT * 
    FROM rental 
    WHERE MONTH(rental_date) = {month} AND YEAR(rental_date) = {year}
    """ 
    return pd.read_sql(query, engine) # return a DataFrame with the query selected

adb1 = rentals_month(cnx,5,2005)
adb2 = rentals_month(cnx,6,2005)



#### 3. Develop a Python function called rental_count_month that takes the DataFrame provided by rentals_month as input along with the month and year and returns a new DataFrame containing the number of rentals made by each customer_id during the selected month and year.

The function should also include the month and year as parameters and use them to name the new column according to the month and year, for example, if the input month is 05 and the year is 2005, the column name should be "rentals_05_2005".

Hint: Consider making use of pandas groupby()

In [54]:
def rental_count_month(data_frame, month, year):
    # Agrupar por customer_id y contar el número de alquileres
    rental_counts = data_frame.groupby('customer_id').size().reset_index(name='rental_count')

    # Crear el nombre de la columna basado en el mes y el año
    column_name = f'rentals_{str(month).zfill(2)}_{str(year)}'

    # Agregar la columna con el número de alquileres al DataFrame original
    data_frame_with_counts = pd.merge(data_frame, rental_counts, on='customer_id')

    # Crear el nuevo DataFrame solamente con la columna de customer_id y el número de alquileres
    result_data_frame = data_frame_with_counts[['customer_id', 'rental_count']].drop_duplicates()

    # Renombrar la columna para el mes y año seleccionados
    result_data_frame.rename(columns={'rental_count': column_name}, inplace=True)

    # Ordenar el DataFrame por la columna de alquileres
    result_data_frame.sort_values(by=column_name, ascending=False, inplace=True)

    return result_data_frame

adb1_1 = rental_count_month(adb1,5,2005)
adb2_1 = rental_count_month(adb2,5,2005)




#### 4. Create a Python function called compare_rentals that takes two DataFrames as input containing the number of rentals made by each customer in different months and years. The function should return a combined DataFrame with a new 'difference' column, which is the difference between the number of rentals in the two months.

In [50]:
def compare_rentals(df1, df2):
    # Merge the two DataFrames on 'customer_id'
    combined_df = pd.merge(df1, df2, on='customer_id', suffixes=('_1', '_2'), how='outer')

    # Replace NaN values with 0
    combined_df['rental_count_1'].fillna(0, inplace=True)
    combined_df['rental_count_2'].fillna(0, inplace=True)

    # Calculate the difference between the two rental counts
    combined_df['difference'] = combined_df['rental_count_1'] - combined_df['rental_count_2']

    return combined_df

compare_rentals(adb1_1,adb2_1)

KeyError: 'rental_count_1'